In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = r'D:/NER_CaseStudy/ner_dataset.csv'
df = pd.read_csv(file_path, encoding='cp1252')

In [3]:
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [4]:
# Fill forward the 'Sentence #' column to ensure every word has a sentence number
df['Sentence #'] = df['Sentence #'].fillna(method='ffill')

# Drop the POS column as it's not needed for this task
df = df.drop(columns=['POS'])

# Ensure that all 'Word' entries are strings
df['Word'] = df['Word'].astype(str)

# Group words and their tags by sentence number
sentences = df.groupby('Sentence #').apply(lambda x: (x['Word'].tolist(), x['Tag'].tolist())).tolist()




C:\Users\RAHUL\AppData\Local\Temp\ipykernel_5236\2569526105.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Sentence #'] = df['Sentence #'].fillna(method='ffill')
C:\Users\RAHUL\AppData\Local\Temp\ipykernel_5236\2569526105.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sentences = df.groupby('Sentence #').apply(lambda x: (x['Word'].tolist(), x['Tag'].tolist())).tolist()


In [5]:
sentences[:3]

[(['Thousands',
   'of',
   'demonstrators',
   'have',
   'marched',
   'through',
   'London',
   'to',
   'protest',
   'the',
   'war',
   'in',
   'Iraq',
   'and',
   'demand',
   'the',
   'withdrawal',
   'of',
   'British',
   'troops',
   'from',
   'that',
   'country',
   '.'],
  ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-geo',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-geo',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-gpe',
   'O',
   'O',
   'O',
   'O',
   'O']),
 (['Iranian',
   'officials',
   'say',
   'they',
   'expect',
   'to',
   'get',
   'access',
   'to',
   'sealed',
   'sensitive',
   'parts',
   'of',
   'the',
   'plant',
   'Wednesday',
   ',',
   'after',
   'an',
   'IAEA',
   'surveillance',
   'system',
   'begins',
   'functioning',
   '.'],
  ['B-gpe',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-tim',
   'O',
   'O',
   'O',
   'B-org',
   'O',
   'O',
   'O',
  

In [6]:
from sklearn.model_selection import train_test_split

# Split the data into training, validation, and test sets
train_data, test_data = train_test_split(sentences, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# Display the sizes of the splits
len(train_data), len(val_data), len(test_data)

(28775, 9592, 9592)

In [7]:
def word2features(sentence, index):
    """
    Extract features for a given word in a sentence.

    Parameters:
    sentence (list of str): The list of words in the sentence.
    index (int): The index of the word to extract features for.

    Returns:
    dict: A dictionary of features for the word.
    """
    word = sentence[index]
    features = {
        'bias': 1.0,
        'word.lower()': word.casefold(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'word.isdigit()': word.isdecimal(),
    }
    if index > 0:
        word1 = sentence[index - 1]
        features.update({
            '-1:word.lower()': word1.casefold(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True  # Beginning of Sentence

    if index < len(sentence) - 1:
        word1 = sentence[index + 1]
        features.update({
            '+1:word.lower()': word1.casefold(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True  # End of Sentence

    return features

def sentence2features(sentence):
    """
    Extract features for all words in a sentence.

    Parameters:
    sentence (list of str): The list of words in the sentence.

    Returns:
    list of dict: A list of dictionaries containing features for each word.
    """
    return [word2features(sentence, i) for i in range(len(sentence))]

def sentence2labels(sentence_tags):
    """
    Extract labels for all words in a sentence.

    Parameters:
    sentence_tags (list of str): The list of tags in the sentence.

    Returns:
    list of str: A list of tags.
    """
    return sentence_tags

def sentence2tokens(sentence):
    """
    Extract tokens for all words in a sentence.

    Parameters:
    sentence (list of str): The list of words in the sentence.

    Returns:
    list of str: A list of words.
    """
    return sentence

In [8]:
X_train = [sentence2features(words) for words, tags in train_data]
y_train = [sentence2labels(tags) for words, tags in train_data]

X_val = [sentence2features(words) for words, tags in val_data]
y_val = [sentence2labels(tags) for words, tags in val_data]

X_test = [sentence2features(words) for words, tags in test_data]
y_test = [sentence2labels(tags) for words, tags in test_data]

In [9]:
!pip install sklearn_crfsuite 

In [10]:
from sklearn.metrics import classification_report
import nltk
import scipy
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_classification_report

In [11]:
%%time
# Step 4: Baseline model (CRF)
crf = CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=False)
crf.fit(X_train, y_train)

# Evaluate the baseline model
y_pred = crf.predict(X_test)
print(flat_classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       B-art       0.36      0.11      0.16        94
       B-eve       0.51      0.29      0.37        70
       B-geo       0.86      0.91      0.88      7558
       B-gpe       0.96      0.93      0.95      3142
       B-nat       0.46      0.33      0.38        40
       B-org       0.80      0.71      0.75      4151
       B-per       0.84      0.81      0.82      3400
       B-tim       0.92      0.88      0.90      4077
       I-art       0.23      0.07      0.11        84
       I-eve       0.32      0.11      0.16        65
       I-geo       0.81      0.79      0.80      1462
       I-gpe       0.94      0.48      0.64        33
       I-nat       0.50      0.15      0.24        13
       I-org       0.80      0.78      0.79      3394
       I-per       0.85      0.88      0.86      3406
       I-tim       0.82      0.79      0.80      1251
           O       0.99      0.99      0.99    177590

    accuracy              

In [12]:
labels = list(crf.classes_)
labels

['B-gpe',
 'O',
 'B-geo',
 'B-tim',
 'I-tim',
 'I-geo',
 'B-org',
 'B-per',
 'I-org',
 'I-per',
 'B-eve',
 'I-eve',
 'I-gpe',
 'B-art',
 'I-art',
 'B-nat',
 'I-nat']

In [13]:
%%time
from sklearn_crfsuite.metrics import flat_f1_score
y_pred = crf.predict(X_test)
flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

CPU times: total: 2.17 s
Wall time: 2.27 s


0.9689074736708978

The f1_score is dominated by O entities in tag. <br> Lets remove it and then calculate flat_f1_score

In [14]:
labels = list(crf.classes_)
labels.remove('O')


In [15]:
%%time
y_pred = crf.predict(X_test)
flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

CPU times: total: 2.08 s
Wall time: 2.21 s


0.8411981934787471

## Hyperparameter tuning

In [16]:
labels.insert(2, 'O')
labels

['B-gpe',
 'B-geo',
 'O',
 'B-tim',
 'I-tim',
 'I-geo',
 'B-org',
 'B-per',
 'I-org',
 'I-per',
 'B-eve',
 'I-eve',
 'I-gpe',
 'B-art',
 'I-art',
 'B-nat',
 'I-nat']

In [17]:
%%time
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score
import scipy.stats

# define fixed parameters and parameters to search
crf = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),             # Coefficient for L1 regularization
    'c2': scipy.stats.expon(scale=0.05),            # Coefficient for L2 regularization
}

# use the same metric for evaluation
f1_scorer = make_scorer(flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
CPU times: total: 7min 2s
Wall time: 36min 1s


RandomizedSearchCV(cv=3,
                   estimator=CRF(algorithm='lbfgs',
                                 all_possible_transitions=True,
                                 max_iterations=100),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022504F35240>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000225A6160430>},
                   scoring=make_scorer(flat_f1_score, response_method='predict', average=weighted, labels=['B-gpe', 'B-geo', 'O', 'B-tim', 'I-tim', 'I-geo', 'B-org', 'B-per', 'I-org', 'I-per', 'B-eve', 'I-eve', 'I-gpe', 'B-art', 'I-art', 'B-nat', 'I-nat']),
                   verbose=1)

### Finding the best model paramter

In [18]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.1964049175254215, 'c2': 0.06373671076669477}
best CV score: 0.9676372423689402
model size: 2.33M


## Creating the model with best model paramter

In [19]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(flat_classification_report(
    y_test, y_pred, labels=labels, digits=3
))

              precision    recall  f1-score   support

       B-gpe      0.964     0.931     0.947      3142
       B-geo      0.857     0.909     0.882      7558
           O      0.990     0.994     0.992    177590
       B-tim      0.923     0.875     0.899      4077
       I-tim      0.796     0.788     0.792      1251
       I-geo      0.810     0.793     0.801      1462
       B-org      0.797     0.711     0.751      4151
       B-per      0.837     0.809     0.823      3400
       I-org      0.798     0.784     0.791      3394
       I-per      0.845     0.882     0.863      3406
       B-eve      0.500     0.286     0.364        70
       I-eve      0.304     0.108     0.159        65
       I-gpe      0.895     0.515     0.654        33
       B-art      0.312     0.106     0.159        94
       I-art      0.207     0.071     0.106        84
       B-nat      0.522     0.300     0.381        40
       I-nat      0.500     0.154     0.235        13

    accuracy              

`We can see there has been a significant surge in the precision of individual tags`

In [20]:
# Prepare a list to store transitions and their scores
from collections import Counter
transitions = []

trans_feature = Counter(crf.transition_features_)

transitions_df = pd.DataFrame.from_dict(trans_feature, orient='index').reset_index()
transitions_df = transitions_df.rename(columns={'index':'Transition', 0:'Weight'})
# Sort transitions by score in descending order
transitions_df = transitions_df.sort_values(by="Weight", ascending=False)

# Display the top 10 transitions
top_n = 10
print(transitions_df.head(top_n))

         Transition    Weight
228  (B-nat, I-nat)  7.108852
206  (B-art, I-art)  6.036389
218  (I-art, I-art)  5.928776
194  (I-gpe, I-gpe)  5.709978
39   (B-geo, I-geo)  5.586272
70   (I-tim, I-tim)  5.578686
184  (I-eve, I-eve)  5.356160
120  (B-per, I-per)  5.347107
171  (B-eve, I-eve)  5.286099
102  (B-org, I-org)  5.003399


In [21]:
# Display the bottom 10 transitions
botton_n = -10
print(transitions_df.iloc[botton_n:])

         Transition    Weight
26       (O, I-per) -4.602011
42   (B-geo, I-org) -4.660414
8    (B-gpe, I-org) -4.692706
134  (I-org, B-org) -4.918294
0    (B-gpe, B-gpe) -4.960288
152  (I-per, B-per) -5.763370
25       (O, I-org) -6.148978
21       (O, I-tim) -6.154378
22       (O, I-geo) -6.431635
118  (B-per, B-per) -9.729412


## Test Model against custom  sentences

In [22]:
def extract_features(word):
    # Define your feature extraction logic here
    return {
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[:3]': word[:3],
         'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
    }

In [23]:
# Function to convert a sentence into features
def sentence_to_features(sentence):
    return [extract_features(word) for word in sentence]

# Predict NER tags for the sentence
def predict_tags(crf, sentence):
    features = sentence_to_features(sentence)
    return crf.predict([features])[0]


In [24]:
# Example sentence and true tags
sentence = ["John", "lives", "in", "New", "York"]
true_tags = ["B-PER", "O", "O", "B-LOC", "I-LOC"]

In [25]:
# Get predicted tags
predicted_tags = predict_tags(crf, sentence)

# Print the results
for word, true_tag, pred_tag in zip(sentence, true_tags, predicted_tags):
    print(f"Word: {word}, True Tag: {true_tag}, Predicted Tag: {pred_tag}")


Word: John, True Tag: B-PER, Predicted Tag: B-geo
Word: lives, True Tag: O, Predicted Tag: I-geo
Word: in, True Tag: O, Predicted Tag: I-geo
Word: New, True Tag: B-LOC, Predicted Tag: I-geo
Word: York, True Tag: I-LOC, Predicted Tag: I-geo


In [26]:

# Calculate accuracy
correct_predictions = sum(t == p for t, p in zip(true_tags, predicted_tags))
accuracy = correct_predictions / len(true_tags)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.00


## Improved Model

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertForTokenClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from seqeval.metrics import classification_report
import numpy as np


In [28]:
file_path = r'D:/NER_CaseStudy/ner_dataset.csv'
df = pd.read_csv(file_path, encoding='cp1252')

In [29]:
# Fill forward the 'Sentence #' column to ensure every word has a sentence number
df['Sentence #'] = df['Sentence #'].fillna(method='ffill')

# Drop the POS column as it's not needed for this task
df = df.drop(columns=['POS'])

# Ensure that all 'Word' entries are strings
df['Word'] = df['Word'].astype(str)

# Group by sentence number and collect words and tags
sentences = df.groupby('Sentence #').apply(lambda s: [(w, t) for w, t in zip(s['Word'].values.tolist(), s['Tag'].values.tolist())])


C:\Users\RAHUL\AppData\Local\Temp\ipykernel_5236\519715000.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Sentence #'] = df['Sentence #'].fillna(method='ffill')
C:\Users\RAHUL\AppData\Local\Temp\ipykernel_5236\519715000.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sentences = df.groupby('Sentence #').apply(lambda s: [(w, t) for w, t in zip(s['Word'].values.tolist(), s['Tag'].values.tolist())])


In [30]:
# Split the data into training, validation, and test sets
train_data, test_data = train_test_split(sentences, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2


# Display the sizes of the splits
len(train_data), len(val_data), len(test_data)

(28775, 9592, 9592)

In [31]:
# Tokenize and preserve labels
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

In [32]:
def tokenize_and_preserve_labels(sentence, text_labels):
    """
    Tokenize a sentence and preserve the word-level labels.

    Parameters:
    sentence (list of str): List of words in the sentence.
    text_labels (list of str): List of labels corresponding to each word in the sentence.

    Returns:
    tuple: Tokenized sentence and the corresponding labels.
    """
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        tokenized_sentence.extend(tokenized_word)
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels



In [33]:
def preprocess_data(data):
    """
    Preprocess the data by tokenizing and preserving labels.

    Parameters:
    data (list of list of tuples): List of sentences, where each sentence is a list of tuples (word, label).

    Returns:
    tuple: Encodings and labels for the tokenized data.
    """
    tokenized_texts_and_labels = [tokenize_and_preserve_labels([word for word, label in sentence], [label for word, label in sentence]) for sentence in data]

    tokenized_texts = [token_labels_pair[0] for token_labels_pair in tokenized_texts_and_labels]
    labels = [token_labels_pair[1] for token_labels_pair in tokenized_texts_and_labels]

    encodings = tokenizer(tokenized_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

    return encodings, labels

In [34]:

train_encodings, train_labels = preprocess_data(train_data)
val_encodings, val_labels = preprocess_data(val_data)
test_encodings, test_labels = preprocess_data(test_data)

In [35]:
# Map tags to indices
unique_tags = set(tag for doc in train_labels for tag in doc)
# Returns: dict mapping each unique tag to an index.
tag2idx = {tag: idx for idx, tag in enumerate(unique_tags)}

In [36]:
tag2idx

{'B-art': 0,
 'B-nat': 1,
 'B-per': 2,
 'B-org': 3,
 'O': 4,
 'I-geo': 5,
 'B-tim': 6,
 'B-geo': 7,
 'I-tim': 8,
 'I-eve': 9,
 'B-gpe': 10,
 'I-org': 11,
 'I-per': 12,
 'I-nat': 13,
 'I-art': 14,
 'I-gpe': 15,
 'B-eve': 16}

In [37]:
# Convert labels to tensor
def encode_labels(labels, encodings):
    """
    Encode labels to match the tokenized inputs.

    Parameters:
    labels (list of list of str): List of labels for each sentence.
    encodings (dict): Encodings returned from the tokenizer.

    Returns:
    list of list of int: Encoded labels aligned with the tokenized inputs.
    """
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings['offset_mapping']):
        doc_enc_labels = np.ones(len(doc_offset), dtype=int) * -100
        arr_offset = np.array(doc_offset)

        doc_enc_labels[(arr_offset[:, 0] == 0) & (arr_offset[:, 1] != 0)] = [tag2idx[label] for label in doc_labels]

        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

In [38]:

train_labels = encode_labels(train_labels, train_encodings)
val_labels = encode_labels(val_labels, val_encodings)
test_labels = encode_labels(test_labels, test_encodings)

In [39]:
# Convert Encoding to tensors
train_encodings = {key: torch.tensor(val) for key, val in train_encodings.items()}
val_encodings = {key: torch.tensor(val) for key, val in val_encodings.items()}
test_encodings = {key: torch.tensor(val) for key, val in test_encodings.items()}


In [40]:
train_encodings

{'input_ids': tensor([[  101,  2169,  2021,  ...,     0,     0,     0],
         [  101,  1109,  7640,  ...,     0,     0,     0],
         [  101,  1262,  1375,  ...,     0,     0,     0],
         ...,
         [  101,  1109, 20857,  ...,     0,     0,     0],
         [  101,  1124,  1163,  ...,     0,     0,     0],
         [  101,  1987,   119,  ...,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'offset_mapping': tensor([[[0, 0],
          [0, 7],
          [0, 6],
          ...,
          [0, 0],
          [0, 0],
      

In [41]:
# # Convert labels to tensors

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
test_labels = torch.tensor(test_labels)

In [42]:
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

In [43]:
# Set up data loaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [44]:
# Load pre-trained model
model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(tag2idx))
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.to(device)
optimizer = AdamW(model.parameters(), lr=3e-5, eps=1e-8)

# Set up the learning rate scheduler
epochs = 3
total_steps = len(train_dataset) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)




Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\RAHUL\anaconda3\envs\myenv\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [45]:
from datetime import datetime
curr_timestamp = datetime.now().strftime('%Y-%m-%d_%H_%M')
curr_timestamp

'2024-07-21_20_08'

In [46]:
import os 
os.getcwd()

# Specify the directory on the D drive
save_directory_model = 'D:\\NER_CaseStudy\\ner_model'+curr_timestamp
save_directory_tokenizer = 'D:\\NER_CaseStudy\\tokenizer'+curr_timestamp

# Create the directory if it doesn't exist
if not os.path.exists(save_directory_model):
    os.makedirs(save_directory_model)

# Create the directory if it doesn't exist
if not os.path.exists(save_directory_tokenizer):
    os.makedirs(save_directory_tokenizer)

In [47]:
%%time

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
    
# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        b_input_ids, b_input_mask, b_labels = batch

        # Move tensors to the GPU
        b_input_ids = b_input_ids.to(device)
        b_input_mask = b_input_mask.to(device)
        b_labels = b_labels.to(device)

        model.zero_grad()

        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss}")

    # Validation loop
    model.eval()
    val_loss = 0

    for batch in val_dataloader:
        b_input_ids, b_input_mask, b_labels = batch

        # Move tensors to the GPU
        b_input_ids = b_input_ids.to(device)
        b_input_mask = b_input_mask.to(device)
        b_labels = b_labels.to(device)

        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Validation loss: {avg_val_loss}")


NVIDIA GeForce RTX 3060 Laptop GPU
Memory Usage:
Allocated: 0.4 GB
Cached:    0.4 GB


C:\Users\RAHUL\anaconda3\envs\myenv\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Average training loss: 0.1907845976658418
Validation loss: 0.13717002308927476
Average training loss: 0.11511767752625963
Validation loss: 0.12744965537451208
Average training loss: 0.08455044302153399
Validation loss: 0.13398817390979578
CPU times: total: 40min 49s
Wall time: 1h 5min 48s


In [49]:
# Save the model
model.save_pretrained(save_directory_model+curr_timestamp)
tokenizer.save_pretrained(save_directory_tokenizer+curr_timestamp)

('D:\\NER_CaseStudy\\tokenizer2024-07-21_20_082024-07-21_20_08\\tokenizer_config.json',
 'D:\\NER_CaseStudy\\tokenizer2024-07-21_20_082024-07-21_20_08\\special_tokens_map.json',
 'D:\\NER_CaseStudy\\tokenizer2024-07-21_20_082024-07-21_20_08\\vocab.txt',
 'D:\\NER_CaseStudy\\tokenizer2024-07-21_20_082024-07-21_20_08\\added_tokens.json',
 'D:\\NER_CaseStudy\\tokenizer2024-07-21_20_082024-07-21_20_08\\tokenizer.json')

In [50]:
idx2tag = {idx: tag for tag, idx in tag2idx.items()}
idx2tag

{0: 'B-art',
 1: 'B-nat',
 2: 'B-per',
 3: 'B-org',
 4: 'O',
 5: 'I-geo',
 6: 'B-tim',
 7: 'B-geo',
 8: 'I-tim',
 9: 'I-eve',
 10: 'B-gpe',
 11: 'I-org',
 12: 'I-per',
 13: 'I-nat',
 14: 'I-art',
 15: 'I-gpe',
 16: 'B-eve'}

In [52]:
# Set the model to evaluation mode
model.eval()

predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_dataloader:
        b_input_ids, b_input_mask, b_labels = batch

        # Move tensors to the appropriate device
        b_input_ids = b_input_ids.to(device)
        b_input_mask = b_input_mask.to(device)
        b_labels = b_labels.to(device)

        # Get model outputs
        outputs = model(b_input_ids, attention_mask=b_input_mask)
        logits = outputs.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        for i in range(len(label_ids)):
            pred_tags = []
            true_tags = []
            for j in range(len(label_ids[i])):
                if label_ids[i][j] != -100:
                    pred_tag_idx = np.argmax(logits[i][j])
                    pred_tags.append(idx2tag[pred_tag_idx])
                    true_tags.append(idx2tag[label_ids[i][j]])

            predictions.append(pred_tags)
            true_labels.append(true_tags)

print(classification_report(true_labels, predictions))

              precision    recall  f1-score   support

         art       0.12      0.04      0.06       166
         eve       0.39      0.26      0.31        80
         geo       0.84      0.90      0.87     12109
         gpe       0.95      0.92      0.94      3525
         nat       0.47      0.15      0.23        59
         org       0.75      0.70      0.72      7295
         per       0.78      0.81      0.79      5579
         tim       0.86      0.85      0.85      4460

   micro avg       0.82      0.83      0.83     33273
   macro avg       0.65      0.58      0.60     33273
weighted avg       0.82      0.83      0.82     33273



In [53]:
# Example sentence and true tags
sentence = ["John", "lives", "in", "New", "York"]
true_tags = ["B-PER", "O", "O", "B-LOC", "I-LOC"]

# Load the pre-trained model and tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(tag2idx))

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenize the input sentence
tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, true_tags)

# Convert the tokenized sentence to input IDs and attention masks
inputs = tokenizer(tokenized_sentence, is_split_into_words=True, return_tensors="pt", padding=True, truncation=True)

# Move tensors to the GPU if available
input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)

# Make predictions
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=2)

# Convert predictions and true labels back to their respective tag names
predictions = predictions.cpu().numpy().flatten()
predicted_tags = [idx2tag[pred] for pred in predictions if pred != -100]

# Output the predicted tags
print("Sentence: ", sentence)
print("Predicted Tags: ", predicted_tags)
print("True Tags: ", true_tags)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sentence:  ['John', 'lives', 'in', 'New', 'York']
Predicted Tags:  ['B-nat', 'I-per', 'I-tim', 'I-gpe', 'I-tim', 'O', 'I-geo']
True Tags:  ['B-PER', 'O', 'O', 'B-LOC', 'I-LOC']
